In [ ]:
from flask import Flask, request, jsonify
import spacy
import torch


def keymatch(text):
     
    # 读取文件
    txt_m = open('../dic/method.txt', 'r', encoding="utf-8")
    txt_rp = open('../dic/researchproblem.txt', 'r', encoding="utf-8")
    # txt_t = open('C:\\Users\\JIBINBIN\\Desktop\\title.txt', 'r', encoding="utf-8")

    l_m = []
    for i in txt_m.readlines():
        l_m.append(i.strip("\n"))

    l_rp = []
    for i in txt_rp.readlines():
        l_rp.append(i.strip("\n"))

    # l_t = []
    # for i in txt_t.readlines():
    #     l_t.append(i.strip("\n"))

    label_list=[]
    for m in l_m:
        try:
            i = text.index(m)
            label_ = 'method'
            start_char = i
            end_char = i+len(m)
#             print('key',i)
            label_list.append({ "label": label_, "start_offset": start_char, "end_offset": end_char })
            break
        except:
            continue
        

    for rp in l_rp:
        try:
            i = text.index(rp)
            label_ = 'problem'
            start_char = i
            end_char = i+len(rp)
            label_list.append({ "label": label_, "start_offset": start_char, "end_offset": end_char })
            break
        except:
            continue
    
    return label_list

def get_method(output_, text):
    output = np.array(output_)
    m_b = np.where(output==1)[0].tolist()
    m_e = np.where(output==2)[0].tolist()
    meth_idx=[]
    if len(p_b)>0: 
        for b in p_b:
            for i in range(b,len(output_)):
                if i==2:
                    continue
                else:
                    e = i-1
                    meth_idx.append([b,e])
                    break
    return meth_idx

def get_problem(output_, text):
    output = np.array(output_)
    p_b = np.where(output==3)[0].tolist()
    p_e = np.where(output==4)[0].tolist()
    pro_idx=[]
    if len(p_b)>0: 
        for b in p_b:
            for i in range(b,len(output_)):
                if i==4:
                    continue
                else:
                    e = i-1
                    pro_idx.append([b,e])
                    break
    return pro_idx

def model_match(text):
    path = './Model/bert_bilstm_crf_1.pkl'
    trained_model = torch.load(path)
    
    pretrained_weights="../bert-base/bert-base-chinese/"
    model = BertModel.from_pretrained(pretrained_weights)
    tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
    
    input_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0)
    outputs = model(input_ids)
    sequence_in = outputs[0][0][1:len(sentence)+1]
    sequence_in = sequence_in.clone().detach().requires_grad_(True)
    _, tag_seq = bert_model(sequence_in)

    METHOD = get_method(tag_seq,text)
    PROBLEM = get_problem(tag_seq,text)
    
    label_list = []
    for p in PROBLEM:
        label_list.append({ "label": 'problem', "start_offset": p[0], "end_offset": p[1] })
    for m in METHOD:
        label_list.append({ "label": 'method', "start_offset": m[0], "end_offset": m[1] })
    


api = Flask(__name__)
@api.route("/predict", methods=["POST"])
def predict():
    # 获取text参数
    text = request.form.get("text") or request.json.get("text") or request.values.get("text")
    
    #模型预测
#     nlp = spacy.load("./model/v2/model-best")
#     ent_label_list = []
    print('text:',text)
#     ent_label_list = keymatch(text)
    ent_label_list = model_match(text)
    
#     for ent in doc.ents:
#         ent_label_list.append({ "label": ent.label_, "start_offset": ent.start_char, "end_offset": ent.end_char })
    print('ent_label_list:',ent_label_list)
    return jsonify(ent_label_list)

if __name__ == "__main__":
    api.run("0.0.0.0", port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)
192.168.0.168 - - [18/Mar/2023 14:30:53] "POST /predict HTTP/1.1" 200 -


text: 基于潜在因子多样性的非负矩阵分解协同过滤模型
key 16
ent_label_list: [{'label': 'method', 'start_offset': 16, 'end_offset': 20}]


192.168.0.168 - - [18/Mar/2023 14:30:58] "POST /predict HTTP/1.1" 200 -


text: 面向稀疏数据的物品相似度和评分预测算法研究
ent_label_list: []


...

text: 结合自监督学习的图神经网络会话推荐
key 8
ent_label_list: [{'label': 'method', 'start_offset': 8, 'end_offset': 13}, {'label': 'research_problem', 'start_offset': 13, 'end_offset': 17}]


192.168.0.168 - - [18/Mar/2023 20:22:59] "POST /predict HTTP/1.1" 200 -


text: 基于可信预测值的协同过滤推荐算法
key 8
ent_label_list: [{'label': 'method', 'start_offset': 8, 'end_offset': 12}]


192.168.0.168 - - [18/Mar/2023 20:23:00] "POST /predict HTTP/1.1" 200 -


text: 结合自监督学习的图神经网络会话推荐
key 8
ent_label_list: [{'label': 'method', 'start_offset': 8, 'end_offset': 13}, {'label': 'research_problem', 'start_offset': 13, 'end_offset': 17}]
